In [1]:
# !curl -O https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
# !unzip -q kagglecatsanddogs_3367a.zip
# !ls PetImages

In [2]:
import re
from glob import glob

import numpy as np
from scipy import stats
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
image_size = (180, 180)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

Found 23410 files belonging to 2 classes.
Using 18728 files for training.
Found 23410 files belonging to 2 classes.
Using 4682 files for validation.


In [4]:
train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)

In [5]:
model_re = re.compile('baseline_(\d+).h5')
models = sorted(glob('models/baseline_*.h5'), key=lambda x: int(model_re.findall(x)[0]))

In [6]:
class SeModel:
    def __init__(self, model_pathes):
        self.model = list(map(keras.models.load_model, model_pathes))
        
    def predict(self, f):
        return np.concatenate(list(map(lambda m: m.predict(f), self.model)), axis=1)

In [7]:
model = keras.models.load_model(models[-1])

In [8]:
se_model_last10 = SeModel(models[-10:])

In [9]:
indeces = 25, 30, 35, 38, 40, 43, 45, 47, 48, 49
se_model_last_selected = SeModel([models[i] for i in indeces])

In [10]:
label = list()
predict = None
se_predict_last10 = None
se_predict_selected = None

for i, j in val_ds:
    if predict is None:
        predict = model.predict(i)
        se_predict_last10 = se_model_last10.predict(i)
        se_predict_selected = se_model_last_selected.predict(i)
    else:
        predict = np.concatenate([predict, model.predict(i)])
        se_predict_last10 = np.concatenate([se_predict_last10, se_model_last10.predict(i)])
        se_predict_selected = np.concatenate([se_predict_selected, se_model_last_selected.predict(i)])
    label.extend(list(map(int, j)))

## Baseline

In [11]:
accuracy_score(list(map(lambda x:x>0.5, predict)), label)

0.9348568987612131

## Take the last 10 epoches

In [12]:
se_predict_last10 = np.array(stats.mode(se_predict_last10>0.5, axis=1))[0, :, :]
accuracy_score(se_predict_last10, label)

0.9483126868859462

## Select 10 snapshots

In [13]:
se_predict_selected = np.array(stats.mode(se_predict_selected>0.5, axis=1))[0, :, :]
accuracy_score(se_predict_selected, label)

0.9463904314395557